In [30]:
from urllib.request import urlopen
import gzip
import pandas as pd
import re
import time

In [31]:
def crawl_race_stats(zipcode):
    try:
        url='https://statisticalatlas.com/zip/'+str(zipcode)+'/Race-and-Ethnicity'
        html = urlopen(url)
        stream = html.read()

        source = str(gzip.decompress(stream))[2:-1]
        race_figure=source.split('<a href="#figure/race-and-ethnicity"')[1].split('<h4 class="figure-title">')[0]
        races=re.findall('<text f[^>]+>([A-Z][^<]+)<\/text>', race_figure, re.DOTALL)
        percent=re.findall('<text f[^>]+>([^>]+%)<\/text>', race_figure, re.DOTALL)
        return dict(zip(races, percent))
    except:
        return ('NA','NA')

In [32]:
crawl_race_stats(89149), crawl_race_stats(8914669)

({'Asian': '7.2%',
  'Black': '11.1%',
  'Hispanic': '14.1%',
  'Mixed': '2.8%',
  'Other': '1.1%',
  'White': '63.7%'},
 ('NA', 'NA'))

In [33]:
# download.geonames.org/export/zip/allCountries.zip

data_path='dataset/'
coords = pd.read_csv(data_path+'allCountries.txt', sep="\t", header=None)
coords = coords[[0,9,10]]
coords[9]  = [int(x*10)/10 for x in coords[9]]
coords[10] = [int(x*10)/10 for x in coords[10]]
coords=coords.applymap(str)
keys=coords[9]+'$'+coords[10]
values=coords[0]
check=dict(zip(keys,values))

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
data_path='dataset/'
with open(data_path+'business.json', 'rb') as f:
    yelp = f.readlines()
yelp=''.join(str(yelp))
columns=["business_id", "name", "city", "state", "postal_code", "latitude", "longitude", "categories"]
data=pd.DataFrame(columns=columns)
for col in columns:
    data[col]=re.findall('"'+col+'": [\[]*(.*?["]*[\]]*["]*[^:]+),', yelp)
    data[col]=data[col].str.replace('"', '').str.replace("\\'s",'s').str.replace("\\","").str.replace('null', '-999')
    if col == 'latitude' or col == 'longitude':
        data[col]=data[col].astype(float)
data['intlat']  = [str(int(x*10)/10) for x in data['latitude']]
data['longlat'] = [str(int(x*10)/10) for x in data['longitude']]
data['key']=data['intlat']+'$'+data['longlat']
data[:3]    

,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.330690,-111.978599,"Dentists, General Dentistry, Health & Medical,...",33.3,-111.9,33.3$-111.9
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.291685,-80.104900,"Hair Stylists, Hair Salons, Mens Hair Salons, ...",40.2,-80.1,40.2$-80.1
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.524903,-112.115310,"Departments of Motor Vehicles, Public Services...",33.5,-112.1,33.5$-112.1


In [35]:
country=[]
for key in data['key']:
    try:
        country.append(check[key])
    except:
        country.append('')
data['country']=country
print('Fehlende: '+str(len(data[data['country']==''])))

Fehlende: 2941


In [36]:
miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

In [37]:
# download.geonames.org/export/zip/allCountries.zip

data_path='dataset/'
codes = pd.read_csv(data_path+'allCountries.txt', sep="\t", header=None)
codes=codes[[0,1,2,3,4,5]].applymap(str)
codes.columns=['country','postal_code','district','state','state_short','county']

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
# quick handle
codes['key']=codes['postal_code']+'$'+codes['state_short']+'$'+codes['district']
keys=codes['key'].tolist()
values=codes['country'].tolist()
check=dict(zip(keys,values))

codes['short']=codes['postal_code']+'$'+codes['state_short']
keys=codes['short'].tolist()
short=dict(zip(keys,values))

codes['short2']=codes['postal_code']+'$'+codes['district']
keys=codes['short2'].tolist()
short2=dict(zip(keys,values))

def check_zip(postal, state, city):
    postal=str(postal).strip().split(' ')[0]  
    
    #quick
    try:
        return check[postal+'$'+state+'$'+city]
    
    #quick
    except:
        try:
            return short[postal+'$'+state]
        #super-slow
        except:
            try:
                return short2[postal+'$'+city]
            except:  
                return ''
            
            #super-slow
            """
            match=codes[codes['postal_code']==postal]
            match=match[match['state_short']==state]
            match=match[match['district'].str.contains(city)]
            match=list(set(match['country'].tolist()))
            l=len(match)
            if l==0:
                return 'not found'
            elif l==1:
                return match[0]
            else:
                return 'multiple results'
            """

In [39]:
check_zip(85044,'AZ', 'Phoenix'),check_zip('M4M 3A6','ON','Toronto'),check_zip(70567,'BW','Stuttgart')

('US', 'CA', 'DE')

In [40]:
country=[]
for index, row in miss.iterrows():
    country.append(check_zip(row['postal_code'], row['state'], row['city']))

In [41]:
miss['country']=country
data=pd.concat([data,miss])

data['intlat2']  = [str(int(x)) for x in data['latitude']]
data['longlat2'] = [str(int(x)) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 27


In [45]:
country=[]
for c in miss['coord2']:
    try:
        country.append(check[c])
    except:
        country.append('')

In [48]:
miss['country']=country
data=pd.concat([data,miss])

data['intlat2']  = [str(int(x)) for x in data['latitude']]
data['longlat2'] = [str(int(x)) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 10


In [50]:
miss

,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key,country,intlat2,longlat2,coord2
17025,W2iWAyXd48iRkCKWOmZZuw,Viu00f1amar,Casablanca,VS,2480000,-33.353150,-71.357290,"Food, Wineries, Arts & Entertainment]",-33.3,-71.3,-33.3$-71.3,,-33,-71,-33$-71
19121,op_1wVF_vxbjnp_gBW9d1Q,Bodegas RE,Casablanca,VS,2340000,-33.226921,-71.467934,"Arts & Entertainment, Wineries, Food]",-33.2,-71.4,-33.2$-71.4,,-33,-71,-33$-71
44939,VcM3_Va-SfkM35XTAV1Idw,Emiliana,Casablanca,VS,,-33.401111,-71.115401,"Food, Wineries, Beer, Wine & Spirits, Arts & E...",-33.4,-71.1,-33.4$-71.1,,-33,-71,-33$-71
90421,1yQUqh3_h1IOrXZmb4CBFw,TriBeCa,Edinburgh,EDH,EH10 4HG,89.999314,-142.466650,"Restaurants, Breakfast & Brunch, American (Tra...",89.9,-142.4,89.9$-142.4,,89,-142,89$-142
123335,YSEW66iMAheOH1x6YYZEPA,Viu00f1a Veramonte,Casablanca,VS,,-33.370547,-71.294938,"Food, Cafes, Restaurants, Wine Tasting Room, C...",-33.3,-71.2,-33.3$-71.2,,-33,-71,-33$-71
124753,MImmd7Y7K3kCAP0YG05hOQ,Macerado Restaurante,Casablanca,VS,2480000,-33.327618,-71.394905,Restaurants],-33.3,-71.3,-33.3$-71.3,,-33,-71,-33$-71
142468,yyF9HvCBuaTa8fMnpN-2-A,Viu00f1a Indu00f3mita,Casablanca,VS,,-33.316431,-71.400177,],-33.3,-71.4,-33.3$-71.4,,-33,-71,-33$-71
154225,vBxK_MAGuy8eWL_CCfUCUQ,Lone Star,George Town,CHE,CW12 4EU,19.327681,-81.381087,"Nightlife, Bars, Restaurants, Tex-Mex, Sports ...",19.3,-81.3,19.3$-81.3,,19,-81,19$-81
159672,6S3sGf8PC5dymj7VI37cuA,Al Settimo Arte - La Carpe,Saint-sauveur,WHT,7912,50.708270,3.598120,"Pizza, Italian, Restaurants]",50.7,3.5,50.7$3.5,,50,3,50$3
174465,rQY4dgr_juFUm4znznGmLQ,Tanino,Casablanca,VS,,-33.317491,-71.434989,"Restaurants, Argentine, Signature Cuisine]",-33.3,-71.4,-33.3$-71.4,,-33,-71,-33$-71


In [51]:
data[:10]

,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key,country,intlat2,longlat2,coord2
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.330690,-111.978599,"Dentists, General Dentistry, Health & Medical,...",33.3,-111.9,33.3$-111.9,US,33,-111,33$-111
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.291685,-80.104900,"Hair Stylists, Hair Salons, Mens Hair Salons, ...",40.2,-80.1,40.2$-80.1,US,40,-80,40$-80
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.524903,-112.115310,"Departments of Motor Vehicles, Public Services...",33.5,-112.1,33.5$-112.1,US,33,-112,33$-112
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,Tempe,AZ,85282,33.383147,-111.964725,"Sporting Goods, Shopping]",33.3,-111.9,33.3$-111.9,US,33,-111,33$-111
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,Cuyahoga Falls,OH,44221,41.119535,-81.475690,"American (New), Nightlife, Bars, Sandwiches, A...",41.1,-81.4,41.1$-81.4,US,41,-81,41$-81
5,o9eMRCWt5PkpLDE0gOPtcQ,Messina,Stuttgart,BW,70567,48.727200,9.147950,"Italian, Restaurants]",48.7,9.1,48.7$9.1,US,48,9,48$9
6,kCoE3jvEtg6UVz5SOD3GVw,BDJ Realty,Las Vegas,NV,89128,36.207430,-115.268460,"Real Estate Services, Real Estate, Home Servic...",36.2,-115.2,36.2$-115.2,US,36,-115,36$-115
7,OD2hnuuTJI9uotcKycxg1A,Soccer Zone,Las Vegas,NV,89128,36.197484,-115.249660,"Shopping, Sporting Goods]",36.1,-115.2,36.1$-115.2,US,36,-115,36$-115
8,EsMcGiZaQuG1OOvL9iUFug,Any Given Sundae,Wexford,PA,15090,40.615102,-80.091349,"Coffee & Tea, Ice Cream & Frozen Yogurt, Food]",40.6,-80.0,40.6$-80.0,US,40,-80,40$-80
9,TGWhGNusxyMaA4kQVBNeew,Detailing Gone Mobile,Henderson,NV,89014,36.055825,-115.046350,"Automotive, Auto Detailing]",36.0,-115.0,36.0$-115.0,US,36,-115,36$-115
